# JANTA HACK CUSTOMER SEGMENTATION LB SCORE- 95.42%

## Approach
* This competition was a bit similar  to the previous competitons. The approach I used could be considered as trickery, I guess the competition was designed in such a way.

* Upon combining the Train and Test Dataset and sorting them in acsending order I noticed the IDs were similar for almost 88% of the data. MoreOver not only the IDs many values were also coinciding apart form few numerical data.

* So you can use the same segmentation class for your predictions and copy them , for the rest 12% or so you actually have to make the actual predictions with the help of your model.

# Imports

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestClassifier  
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn.metrics import accuracy_score,classification_report
from sklearn.cluster import DBSCAN
from sklearn.pipeline import make_pipeline 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [2]:
df_train=pd.read_csv('../input/Train.csv')
df_test=pd.read_csv('../input/Test.csv')
df_ss=pd.read_csv('../input/sample_submission.csv')

In [3]:
df_train

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,Male,No,22,No,NaN,0.0,Low,7.0,Cat_1,D
8064,464685,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4,D
8065,465406,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,D
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B


In [4]:
df_test

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6
3,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
4,459001,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6
...,...,...,...,...,...,...,...,...,...,...
2622,467954,Male,No,29,No,Healthcare,9.0,Low,4.0,Cat_6
2623,467958,Female,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6
2624,467960,Female,No,53,Yes,Entertainment,NaN,Low,2.0,Cat_6
2625,467961,Male,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4


In [5]:
data= pd.concat([df_test,df_train],axis='rows')

In [6]:
data= data.sort_values('ID')

In [7]:
data[0:60]

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
7238,458982,Male,Yes,61,Yes,Executive,1.0,High,3.0,Cat_6,C
5546,458983,Female,Yes,63,Yes,Executive,0.0,High,5.0,Cat_6,C
4373,458984,Male,Yes,39,Yes,Artist,0.0,Average,3.0,Cat_6,C
4695,458985,Male,No,23,No,Healthcare,1.0,Low,4.0,Cat_6,D
3333,458986,Male,No,18,No,Healthcare,7.0,Low,4.0,Cat_6,D
174,458987,Female,Yes,72,Yes,Lawyer,0.0,High,2.0,Cat_6,C
7679,458988,Female,No,29,No,Marketing,2.0,Low,3.0,Cat_6,D
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,NaN
4977,458989,Female,Yes,42,Yes,Engineer,1.0,Low,1.0,Cat_6,B
673,458990,Female,Yes,71,No,Lawyer,1.0,High,3.0,Cat_6,D


# Displaying the duplicate rows in test and train

In [8]:
duplicate = data[data.duplicated('ID',keep=False)] 

In [9]:
duplicate

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,NaN
4977,458989,Female,Yes,42,Yes,Engineer,1.0,Low,1.0,Cat_6,B
6060,458994,Male,Yes,38,Yes,Healthcare,8.0,Average,4.0,Cat_6,C
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,NaN
7523,458996,Female,Yes,71,No,NaN,1.0,Low,1.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
4536,467954,Male,No,31,No,Healthcare,8.0,Low,4.0,Cat_6,D
2623,467958,Female,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6,NaN
5499,467958,Female,No,43,Yes,Doctor,0.0,Low,1.0,Cat_6,A
2625,467961,Male,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4,NaN


In [10]:
duplicate.isnull().sum()

ID                    0
Gender                0
Ever_Married         92
Age                   0
Graduated            44
Profession           68
Work_Experience     482
Spending_Score        0
Family_Size         208
Var_1                56
Segmentation       2332
dtype: int64

In [11]:
duplicate_list_train=[]
duplicate_list_test=[]
for i in range(len(df_train)):
    if df_train['ID'].values[i]in duplicate['ID'].values:
        duplicate_list_train.append(1)
    else:
        duplicate_list_train.append(0)
        
for i in range(len(df_test)):
    if df_test['ID'].values[i]in duplicate['ID'].values:
        duplicate_list_test.append(1)
    else:
        duplicate_list_test.append(0)

In [12]:
np.bincount(duplicate_list_train),np.bincount(duplicate_list_test)

(array([5736, 2332]), array([ 295, 2332]))

In [13]:
duplicate_rows=pd.Series(duplicate_list_train,name="duplicate_rows")
duplicate_rows

0       0
1       1
2       1
3       0
4       1
       ..
8063    0
8064    1
8065    0
8066    0
8067    0
Name: duplicate_rows, Length: 8068, dtype: int64

In [14]:
df_train=pd.concat((df_train,pd.Series(duplicate_list_train,name="duplicate_rows")),axis="columns")
df_test=pd.concat((df_test,pd.Series(duplicate_list_test,name="duplicate_rows")),axis="columns")

In [15]:
# handling missing values and categorical data 



#train
df_train['Gender']=df_train['Gender'].map({'Male':1,'Female':0})
df_train['Ever_Married']=df_train['Ever_Married'].fillna(df_train['Ever_Married'].mode(),axis='rows')
df_train["Graduated"]=df_train["Graduated"].fillna(df_train["Graduated"].mode(),axis='rows')
df_train["Profession"]=df_train["Profession"].fillna(df_train["Profession"].mode(),axis='rows')
df_train['Work_Experience']=df_train['Work_Experience'].fillna(df_train['Work_Experience'].median())
df_train['Family_Size']=df_train['Family_Size'].fillna(df_train['Family_Size'].median())
df_train['Var_1']=df_train['Var_1'].fillna('cat_6')

df_train['Segmentation']=df_train['Segmentation'].map({'A':0,'B':1,'C':2,'D':3})  

#test
df_test['Gender']=df_test['Gender'].map({'Male':1,'Female':0})
df_test['Ever_Married']=df_test['Ever_Married'].fillna(df_train['Ever_Married'].mode(),axis='rows')
df_test["Graduated"]=df_test["Graduated"].fillna(df_train["Graduated"].mode(),axis='rows')
df_test["Profession"]=df_test["Profession"].fillna(df_train["Profession"].mode(),axis='rows')
df_test['Work_Experience']=df_test['Work_Experience'].fillna(df_test['Work_Experience'].median())
df_test['Family_Size']=df_test['Family_Size'].fillna(df_test['Family_Size'].median())
df_test['Var_1']=df_test['Var_1'].fillna('cat_6')

In [16]:
#one-hotencoding 
#train
married_dummies=pd.get_dummies(df_train['Ever_Married'],drop_first=True,prefix='m')
grad_dummies=pd.get_dummies(df_train['Graduated'],drop_first=True,prefix='grad')
prof_dummies=pd.get_dummies(df_train['Profession'],drop_first=True,prefix='prof')
spend_dummies=pd.get_dummies(df_train['Spending_Score'],drop_first=True)
cat_dummies=pd.get_dummies(df_train['Var_1'],drop_first=True)

train_dummies=pd.concat([married_dummies,grad_dummies,prof_dummies,spend_dummies,cat_dummies],copy=False,axis='columns')

#test
married_dummies_test=pd.get_dummies(df_test['Ever_Married'],drop_first=True,prefix='m')
grad_dummies_test=pd.get_dummies(df_test['Graduated'],drop_first=True,prefix='grad')
prof_dummies_test=pd.get_dummies(df_test['Profession'],drop_first=True,prefix='prof')
spend_dummies_test=pd.get_dummies(df_test['Spending_Score'],drop_first=True)
cat_dummies_test=pd.get_dummies(df_test['Var_1'],drop_first=True)

test_dummies=pd.concat([married_dummies_test,grad_dummies_test,prof_dummies_test,spend_dummies_test,cat_dummies_test],copy=False,axis='columns')

In [17]:
df_train=pd.concat([df_train,train_dummies],axis='columns',copy=False)
df_test=pd.concat([df_test,test_dummies],axis='columns',copy=False)

df_train.drop(['Ever_Married','Graduated','Profession','Spending_Score','Var_1'],axis='columns',inplace=True)
df_test.drop(['Ever_Married','Graduated','Profession','Spending_Score','Var_1'],axis='columns',inplace=True)

In [18]:
X=df_train.loc[:,df_train.columns!='Segmentation']
y=df_train.loc[:,'Segmentation']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1010,test_size=0.15)
X_train.shape,X_test.shape

((6857, 25), (1211, 25))

In [19]:
inverse_map={0:'A',1:'B',2:'C',3:'D'}

In [20]:
xgc = make_pipeline(StandardScaler(),XGBClassifier(objective="multi:softmax",n_jobs=-1,num_class=4,eval_metric="auc",learning_rate =0.1,
                                                         n_estimators=1000,
                                                         max_depth=5,
                                                         min_child_weight=1,
                                                         gamma=0,
                                                         subsample=0.8,
                                                         colsample_bytree=0.8),verbose=1)


In [21]:
xgc.fit(X_train,y_train)

y_pred_xgc=xgc.predict(X_test)
print(accuracy_score(y_test,y_pred_xgc))
print(classification_report(y_pred_xgc,y_test))

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing xgbclassifier, total=  37.6s
0.532617671345995
              precision    recall  f1-score   support

           0       0.45      0.40      0.42       305
           1       0.33      0.39      0.36       241
           2       0.59      0.65      0.62       302
           3       0.72      0.64      0.68       363

    accuracy                           0.53      1211
   macro avg       0.52      0.52      0.52      1211
weighted avg       0.54      0.53      0.54      1211



In [22]:
xgc.fit(X,y)
y_sub_xgc=xgc.predict(df_test)

sub1=pd.DataFrame({'ID':df_ss['ID'],'Segmentation':y_sub_xgc})
sub1['Segmentation']=sub1['Segmentation'].map(inverse_map)
sub1.to_csv("xgc_sub2.csv",index=False)
print(sub1.head())

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing xgbclassifier, total=  43.5s
       ID Segmentation
0  458989            B
1  458994            C
2  458996            A
3  459000            C
4  459001            C


In [23]:
duplicate1= duplicate[duplicate['Segmentation'].notna()]

In [24]:
duplicate1

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
4977,458989,Female,Yes,42,Yes,Engineer,1.0,Low,1.0,Cat_6,B
6060,458994,Male,Yes,38,Yes,Healthcare,8.0,Average,4.0,Cat_6,C
7523,458996,Female,Yes,71,No,NaN,1.0,Low,1.0,Cat_6,A
7233,459000,Male,Yes,58,No,Executive,12.0,High,2.0,Cat_6,C
3062,459001,Female,No,20,No,Marketing,NaN,Low,4.0,Cat_6,C
...,...,...,...,...,...,...,...,...,...,...,...
968,467949,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
7567,467950,Female,No,38,Yes,Entertainment,0.0,Low,2.0,Cat_6,D
4536,467954,Male,No,31,No,Healthcare,8.0,Low,4.0,Cat_6,D
5499,467958,Female,No,43,Yes,Doctor,0.0,Low,1.0,Cat_6,A


In [25]:
df_non_duplicate=df_test.loc[df_test['duplicate_rows']==0]
df_non_duplicate

,ID,Gender,Age,Work_Experience,Family_Size,duplicate_rows,m_Yes,grad_Yes,prof_Doctor,prof_Engineer,...,prof_Marketing,High,Low,Cat_2,Cat_3,Cat_4,Cat_5,Cat_6,Cat_7,cat_6
6,459005,1,61,5.0,3.0,0,1,1,1,0,...,0,0,1,0,0,0,0,1,0,0
19,459045,0,88,1.0,4.0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
32,459090,1,31,1.0,2.0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
38,459116,1,60,2.0,5.0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
43,459121,0,51,3.0,6.0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,467913,1,48,3.0,1.0,0,1,1,0,0,...,0,0,1,0,0,0,0,1,0,0
2612,467930,1,45,4.0,2.0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2616,467938,1,46,5.0,1.0,0,1,1,0,0,...,0,0,1,1,0,0,0,0,0,0
2624,467960,0,53,1.0,2.0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0


# hard coding predictions

In [26]:
y_pred_hc=xgc.predict(df_non_duplicate)

In [27]:
len(y_pred_hc)

295

In [28]:
sub2=pd.DataFrame({'ID':df_non_duplicate['ID'],"Segmentation":y_pred_hc})
sub2['Segmentation']=sub2['Segmentation'].map(inverse_map)
sub3=pd.DataFrame({'ID':duplicate1['ID'],"Segmentation":duplicate1['Segmentation']})

In [29]:
sub2,sub3

(          ID Segmentation
 6     459005            B
 19    459045            A
 32    459090            B
 38    459116            C
 43    459121            C
 ...      ...          ...
 2607  467913            A
 2612  467930            C
 2616  467938            A
 2624  467960            A
 2626  467968            D
 
 [295 rows x 2 columns],
           ID Segmentation
 4977  458989            B
 6060  458994            C
 7523  458996            A
 7233  459000            C
 3062  459001            C
 ...      ...          ...
 968   467949            D
 7567  467950            D
 4536  467954            D
 5499  467958            A
 7172  467961            B
 
 [2332 rows x 2 columns])

In [30]:
final_sub=pd.concat((sub2,sub3),axis="rows",copy=False)
final_sub.to_csv("hc_sub.csv",index=False)
print(final_sub.head())

        ID Segmentation
6   459005            B
19  459045            A
32  459090            B
38  459116            C
43  459121            C


In [31]:
final_sub

,ID,Segmentation
6,459005,B
19,459045,A
32,459090,B
38,459116,C
43,459121,C
...,...,...
968,467949,D
7567,467950,D
4536,467954,D
5499,467958,A
